# Quality standards

## Imports

In [1]:
from weldx.config import Config, QualityStandard
import fs
import yaml
from pathlib import Path

## Introduction

The main purposes of WelDX is to provide a file format for experimental data that is self explanatory and contains all
relevant information about the conducted experiment.
This allows fellow researchers to understand and analyze the data without taking part in the experiment or having access
to the persons who did.
We assure this by introducing quality standards for each type of experiment and every other information that
describes certain aspects of an experiment like coordinate systems, sensors or specimens.
Every time a WelDX file is written or read, it is validated against those standards.

## Schema files

Every standard used by WelDX is defined by schema files.
A schema file is a [YAML](https://yaml.org/)-file that defines the allowed properties of an serializable object.
How such a schema file is generated is not part of this tutorial, but you can find further information regarding this
topic in the official
[documentation of the ASDF standard](https://asdf-standard.readthedocs.io/en/latest/schemas.html) which is the
foundation of a WelDX file.
Furthermore, an overview of all pre-defined schema files can be found on the [Standards](../standard.rst) page of this
documentation.

> todo: fix link

## Installing and using quality standards

The WelDX API comes with a default schema for every object.
So if you save an ASDF file using the WelDX extension you already employ the default WelDX quality standard.
However, the standards we defined might not suit your needs.
Therefore, the WelDX API offers a mechanism to override an arbitrary number of default schemas.
Before we discuss, how a custom standard can be generated, let's assume that you have found and downloaded one.
Each quality standard that was created following the guidelines (Todo: Link) can be installed using pythons package
manager `pip`.
So the first step would be to install the standard as a python package:

~~~ shell
pip install NAME_OF_THE_STANDARD
~~~

Installing the quality standard registers it to the WelDX API so that it knows that the standard exists and where to
find the corresponding data.
If you want to use the standard, you have to activate it first:

~~~ python
from weldx.config import Config

Config.enable_quality_standard(NAME_OF_THE_STANDARD, VERSION_OF_THE_STANDARD)
~~~

The version number is optional.
If you don't provide one, the latest available version will be used.
And that's all there is to know about using standards.

## Creating custom standards

In [2]:
mem_fs = fs.open_fs('mem://')
mem_fs.makedirs("resources/my_organization/manifests")
mem_fs.makedirs("resources/my_organization/schemas")
mem_fs.listdir("")

mem_fs.create("resources/my_organization/manifests/my_standard-1.0.0.yaml")
mem_fs.tree()

`-- resources
    `-- my_organization
        |-- manifests
        |   `-- my_standard-1.0.0.yaml
        `-- schemas


In [3]:
manifest = """
id: http://stsci.edu/asdf/extensions/transform/manifests/transform-1.0.0
extension_uri: http://stsci.edu/asdf/extensions/transform-1.0.0
title: Transform extension 1.0.0
description: |-
  A set of tags for serializing data transforms.
asdf_standard_requirement: 1.0.0
tags:
- uri: "http://weldx.bam.de/schemas/weldx/core/transformations/local_coordinate_system-1.0.0"
  file: "lcs_schema_fancy"
"""

In [4]:
with mem_fs.open("resources/my_organization/manifests/my_standard-1.0.0.yaml", "w") as file:
    file.write(manifest)

In [5]:
with mem_fs.open("resources/my_organization/manifests/my_standard-1.0.0.yaml", "r") as file:
    content = yaml.load(file, Loader=yaml.SafeLoader)
    print(content)

{'id': 'http://stsci.edu/asdf/extensions/transform/manifests/transform-1.0.0', 'extension_uri': 'http://stsci.edu/asdf/extensions/transform-1.0.0', 'title': 'Transform extension 1.0.0', 'description': 'A set of tags for serializing data transforms.', 'asdf_standard_requirement': '1.0.0', 'tags': [{'uri': 'http://weldx.bam.de/schemas/weldx/core/transformations/local_coordinate_system-1.0.0', 'file': 'lcs_schema_fancy'}]}


In [6]:
qs = QualityStandard(Path("resources/my_organization"), mem_fs)